In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from glob import glob

%matplotlib widget

In [ ]:
# Fitting for polynomials
# Injecting a gaussian

def _model_gaussian(x, a, b, d):
    return a * np.exp(-0.5 * ( (x-d) / (b) )**2)
    
def inject_gaussian(times, magnitude, three_sigma_width, zp = 20.44):
    amplitude = _amplitude(magnitude, zp)
    position = np.random.uniform(min(times), max(times))
    sigma_width = three_sigma_width/3
    noise = np.random.normal(loc = 0, scale = 0.05, size = len(times))
    gaussian = _model_gaussian(times, amplitude, sigma_width, position)
    gaussian += noise
    return gaussian
    
def _amplitude(m, zp):
    return 10**((zp - m)/2.5)

def _splitting(mjd):
    gap = np.gradient(mjd)
    gap_inds = np.where(gap > 0.5)[0]
    return gap_inds[1]

def fitting_function(x, y, sigma, order = 2):
    split = _splitting(x)
    
    x0, y0, s0 = sigma_weighted_polyfit(x[:split], y[:split], sigma[:split], order = order)
    x1, y1, s1 = sigma_weighted_polyfit(x[split:], y[split:], sigma[split:], order = order)
    
    x = np.array( x0.tolist() + x1.tolist() )
    y = np.array( y0.tolist() + y1.tolist() )
    s = np.array( s0.tolist() + s1.tolist() )
    
    return x, y, s

def sigma_weighted_polyfit(x, y, sigma, order = 2):
    """
    Fit a polynomial of given order to data with sigma-weighted least squares.

    Parameters:
    - x: array-like, independent variable
    - y: array-like, dependent variable
    - sigma: array-like, uncertainties in y
    - order: int, order of the polynomial

    Returns:
    - y_fit: fitted y values at x
    """
    x = np.asarray(x)
    y = np.asarray(y)
    sigma = np.asarray(sigma)

    mask = np.isfinite(x) & np.isfinite(y) & np.isfinite(sigma) & (sigma > 0)
    if np.sum(mask) < order + 1:
        raise ValueError("Not enough valid data points for the requested polynomial order.")

    x = x[mask]
    y = y[mask]
    sigma = sigma[mask]
    weights = 1.0 / sigma**2

    coeffs = np.polyfit(x, y, deg=order, w=weights)
    y_fit = np.polyval(coeffs, x)

    return x, y_fit, sigma



In [ ]:
folder = '/Users/zgl12/Modules/SN2019vxm/Telescopes_Scripts/TESS/Control_Curves/Original_Lightcurves/'

In [ ]:
files = glob(folder + '*18*psf*.csv')

plt.figure()
for file in files:
    df = pd.read_csv(file)
    df = df[(df['MJD'] < 58801) & (df['MJD'] > 58792) | (df['MJD'] > 58802) & (df['MJD'] < 58812)]
    
    mjd = df.MJD.values
    flux = df.flux.values
    dflux = df.dflux.values
    dflux[dflux < 1] = 1
    zp = df.ZP.values[0]
    
    gaussian = inject_gaussian(mjd, 19, 2, zp = zp)
    
    flux += gaussian
    
    # fit_flux = sigma_weighted_polyfit(mjd, flux, dflux, order = 1)
    fit_mjd, fit_flux, fit_dflux = fitting_function(mjd, flux, dflux, order = 2)
    modified_flux = flux - fit_flux
    
    # xs
    plt.errorbar(fit_mjd, modified_flux, fit_dflux, fmt = '.')
    # plt.axvline(mjd[gap_inds[1]], color = 'r')
    break
plt.show()

